In [13]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','outcome']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.01):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    if total_rule/total_records<0.4:
        return (999,);

    avg_confidence = np.sum(rule_confidences) / count +1*(entropy_D-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof



def bestIndividual(hof):

    def SVM(trainX,trainY,testX,testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY)/testY.shape[0]
        return precision
    maxaccuracy=0.0
    for ind in hof:
        df_train = pd.read_csv('data/horse/train.csv')
        df_test = pd.read_csv('data/horse/test.csv')
        feature_columns = [col for col in df_train.columns if col not in ['id','outcome']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]

        trainX=df_train[selected_features]
        trainY=df_train['outcome']
        testX=df_test[selected_features]
        testY=df_test['outcome']
        score=SVM(trainX,trainY,testX,testY)
        if(score>maxaccuracy):
            maxaccuracy=score
            print(maxaccuracy)
            best_feature=selected_features
            print(best_feature)
    return maxaccuracy,best_feature


if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=50
    n_gen=100
    # read dataframe from csv
    df = pd.read_csv('data/horse/train.csv')
    # encode labels column to numbers
    y = df['outcome']
    # feature_columns_con=['rectal temperature','pulse','respiratory rate',' packed cell volume','total protein']
    feature_columns = [ col for col in df.columns if col not in ['id','outcome']]
    for col in feature_columns:
        df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
    # get accuracy with all features
    feature_columns = [ col for col in df.columns if col not in ['id','outcome']]
    X = df[feature_columns]
    individual = [1 for i in range(len(X.columns))]
    # getFitness(df,'outcome',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'outcome',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'outcome',n_pop, n_gen)

Accuracy with all features: 	(999,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	50    	899.359	2.37269
1  	26    	799.694	2.12232
2  	32    	620.314	2.12232
3  	34    	480.84 	2.12232
4  	37    	241.736	2.09384
5  	34    	221.702	2.09384
6  	27    	161.859	2.01536
7  	30    	121.854	2.01536
8  	34    	261.43 	2.01536
9  	26    	42.0583	2.01536
10 	31    	61.9261	1.98168
11 	28    	141.664	2.00203
12 	26    	2.0797 	2.00203
13 	17    	81.7715	1.98168
14 	26    	21.9475	1.96835
15 	28    	81.7564	1.96835
16 	31    	81.7606	1.96835
17 	25    	41.8675	1.96835
18 	39    	121.622	1.96835
19 	21    	41.8637	1.96835
20 	31    	41.8697	1.96835
21 	24    	81.742 	1.96835
22 	34    	61.8139	1.96835
23 	29    	101.672	1.96835
24 	30    	21.9173	1.96835
25 	20    	101.672	1.96835
26 	28    	41.8496	1.96835
27 	27    	21.9159	1.96835
28 	31    	101.688	1.96835
29 	25    	41.8608	1.96835
30 	25    	41.8604	1.96835
31 	32    	61.7909	1.96835
32 	30    	141.562	1.96835
33 	30    	81.7375	1.96835
34 	33    	21.921 	1.96835
35 	32    	21.911 	1.96835
3

In [14]:
accuracy,individual = bestIndividual(hof)
print(accuracy)
print(individual)

0.97
['surgery', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'pain', 'abdominal distension', 'nasogastric reflux', 'total protein', 'surgical lesion?', '2', 'path']
0.9766666666666667
['surgery', 'Age', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'pain', 'abdominal distension', 'nasogastric reflux', 'total protein', '3', 'path']
0.98
['surgery', 'Age', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'pain', 'abdominal distension', 'nasogastric reflux', 'total protein', 'surgical lesion?', '3', 'path']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.9866666666666667
['surgery', 'Age', 'pulse', 'temperature of extremities', 'peripheral pulse', 'pain', 'abdominal distension', 'total protein', 'surgical lesion?', '2', '3', 'path']
0.99
['surgery', 'Age', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'pain', 'abdominal distension', 'total protein', 'surgical lesion?', '1', '2', '3', 'path']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.9966666666666667
['surgery', 'Age', 'pulse', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'pain', 'peristalsis', 'nasogastric reflux', 'total protein', 'surgical lesion?', '1', '2', '3']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

1.0
['rectal temperature', 'pulse', 'capillary refill time', 'pain', 'peristalsis', ' packed cell volume', 'total protein', '1', 'path']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

1.0
['rectal temperature', 'pulse', 'capillary refill time', 'pain', 'peristalsis', ' packed cell volume', 'total protein', '1', 'path']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv